# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json as json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# data files
data_path = '../WeatherPy/output_data/cities.csv'

# Read the data and filter results
data = pd.read_csv(data_path)
cities_df = pd.DataFrame(data)
cities_df.head()
# Store latitude and longitude in locations
locations = cities_df[["lat", "lon"]]
# Fill humidity values
humidity = cities_df["humid"]
cities_df.head()

,country,city,lat,lon,temp,max_temp,humid,cover,wind
0,ZA,{'bredasdorp'},-34.5322,20.0403,40.32,40.32,93,0,3.44
1,PF,{'tautira'},-17.7333,-149.1500,80.62,80.62,82,72,16.60
2,MU,{'mahebourg'},-20.4081,57.7000,68.29,68.29,100,75,3.44
3,NZ,{'kaitangata'},-46.2817,169.8464,46.20,46.20,84,16,6.89
4,KZ,{'ushtobe'},45.2514,77.9838,66.43,66.43,44,0,5.32


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Make a new DataFrame to drop all rejected cities
nice_df = cities_df.loc[(cities_df["temp"] >= 78) &
                       (cities_df["temp"] <= 85) &
                       (cities_df["wind"] < 9) &
                       (cities_df["cover"] < 5) &
                       (cities_df["humid"] < 80 ) &
                       (cities_df["max_temp"] < 90)].copy()
nice_df = pd.DataFrame(nice_df)                       
nice_df.reset_index(inplace=True)
#len(nice_df)
nice_df.head()

,index,country,city,lat,lon,temp,max_temp,humid,cover,wind
0,45,US,{'green river'},41.5286,-109.4662,84.74,86.18,16,0,8.99
1,84,CK,{'avarua'},-21.2078,-159.7750,80.65,80.65,74,3,5.75
2,92,ES,{'olivenza'},38.6832,-7.1030,81.32,87.21,32,0,7.16
3,219,LY,{'awjilah'},29.1081,21.2869,84.70,84.70,54,0,8.79
4,233,PS,{'gasa'},31.5000,34.4667,81.18,82.38,71,0,3.40


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# geocoordinates
coordinates = nice_df.loc[:,["lat","lon"]]
coordinates_df = pd.DataFrame(coordinates)
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"
# hotel list
hotels = []

for i, row in coordinates_df.iterrows():
    target_coordinates = f"{row['lat']},{row['lon']}"
    # set up parms dictionary
    parms = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using parms dictionary
    try:
        response = requests.get(base_url, params=parms)
        # convert response to json
        response = response.json()
    except KeyError:
        #high level key failure
        print(f"No data returned for coordinates: {target_coordinates}")         
    except:
        #some rando failure
        print(f"No data returned for coordinates: {target_coordinates}")
    else:
        # Print the json (pretty printed)
        #print(json.dumps(response['results'], indent=4, sort_keys=True))
        try:
            hotels.append(response['results'][0]['name'])
            print(response["results"][0]["name"])
        except:
            hotels.append('none')
            print(f"No Hotel name returned for coordinates: {target_coordinates}")

# create dataframe from hotels and cities 
nice_df['hotel'] = hotels
nice_df.head()

Hampton Inn & Suites Green River
The Islander Hotel
Hotel Heredero
No Hotel name returned for coordinates: 29.1081,21.2869
فندق جراند بالاس Grand Palace Hotel
Hotel Beira Rio


,index,country,city,lat,lon,temp,max_temp,humid,cover,wind,hotel
0,45,US,{'green river'},41.5286,-109.4662,84.74,86.18,16,0,8.99,Hampton Inn & Suites Green River
1,84,CK,{'avarua'},-21.2078,-159.7750,80.65,80.65,74,3,5.75,The Islander Hotel
2,92,ES,{'olivenza'},38.6832,-7.1030,81.32,87.21,32,0,7.16,Hotel Heredero
3,219,LY,{'awjilah'},29.1081,21.2869,84.70,84.70,54,0,8.79,none
4,233,PS,{'gasa'},31.5000,34.4667,81.18,82.38,71,0,3.40,فندق جراند بالاس Grand Palace Hotel


In [6]:
drop_index = nice_df[nice_df['hotel'] == 'none'].index
nice_df.drop(drop_index, inplace=True)
nice_df.rename(columns={"hotel": "Hotel Name", "lat": "Lat", "lon": "Lng", "city": "City", "country": "Country"}, inplace=True)
hotel_df = nice_df.copy()
hotel_df.head()

,index,Country,City,Lat,Lng,temp,max_temp,humid,cover,wind,Hotel Name
0,45,US,{'green river'},41.5286,-109.4662,84.74,86.18,16,0,8.99,Hampton Inn & Suites Green River
1,84,CK,{'avarua'},-21.2078,-159.7750,80.65,80.65,74,3,5.75,The Islander Hotel
2,92,ES,{'olivenza'},38.6832,-7.1030,81.32,87.21,32,0,7.16,Hotel Heredero
4,233,PS,{'gasa'},31.5000,34.4667,81.18,82.38,71,0,3.40,فندق جراند بالاس Grand Palace Hotel
5,452,BR,{'itupiranga'},-5.1347,-49.3267,84.52,84.52,39,0,2.01,Hotel Beira Rio


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel_locations)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))